In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [2]:
data = pd.read_csv("dollar_data/dollar_data_SIGNALS.csv")

In [3]:
data

,Unix,Date,Open,High,Low,Close,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483269300,2017-01-01 11:15:00,974.99,975.80,972.50,974.07,-0.215342,3.072177,0.000444,0.031253,-0.398718,-0.390314,-2.757467e-15
1,1483279320,2017-01-01 14:02:00,974.07,978.22,973.20,977.91,0.857599,2.588866,0.007401,0.043285,-0.398718,-0.363255,-2.757467e-15
2,1483287120,2017-01-01 16:12:00,976.77,984.12,976.22,984.12,1.385308,2.705708,0.017140,0.060230,-0.398718,-0.305130,-2.757467e-15
3,1483288740,2017-01-01 16:39:00,984.12,992.00,984.01,992.00,1.748401,2.808421,0.030164,0.082427,-0.398718,-0.272058,-2.757467e-15
4,1483289880,2017-01-01 16:58:00,991.99,995.02,981.57,994.54,0.554800,2.877294,0.041725,0.096336,-0.398718,-0.353234,-2.757467e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
206015,1646099520,2022-03-01 01:52:00,43436.20,43436.20,43271.46,43273.95,-0.800713,0.508509,1.600107,-2.140442,1.490217,-0.403676,-1.200815e+00
206016,1646099880,2022-03-01 01:58:00,43273.95,43325.97,43128.12,43139.57,-0.686825,-0.306142,1.310527,-2.474145,1.847879,-0.403676,-1.193196e+00
206017,1646100600,2022-03-01 02:10:00,43109.61,43174.93,42940.42,43009.25,-0.668321,-1.435721,0.973669,-2.865480,2.543143,-0.403676,-1.076745e+00
206018,1646100960,2022-03-01 02:16:00,42998.08,43247.75,42998.08,43154.82,0.735404,-0.824978,0.804504,-2.737410,-0.398718,0.812617,-9.893302e-01


In [4]:
data[0:20]

,Unix,Date,Open,High,Low,Close,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483269300,2017-01-01 11:15:00,974.99,975.80,972.50,974.07,-0.215342,3.072177,0.000444,0.031253,-0.398718,-0.390314,-2.757467e-15
1,1483279320,2017-01-01 14:02:00,974.07,978.22,973.20,977.91,0.857599,2.588866,0.007401,0.043285,-0.398718,-0.363255,-2.757467e-15
2,1483287120,2017-01-01 16:12:00,976.77,984.12,976.22,984.12,1.385308,2.705708,0.017140,0.060230,-0.398718,-0.305130,-2.757467e-15
3,1483288740,2017-01-01 16:39:00,984.12,992.00,984.01,992.00,1.748401,2.808421,0.030164,0.082427,-0.398718,-0.272058,-2.757467e-15
4,1483289880,2017-01-01 16:58:00,991.99,995.02,981.57,994.54,0.554800,2.877294,0.041725,0.096336,-0.398718,-0.353234,-2.757467e-15
5,1483294500,2017-01-01 18:15:00,994.04,998.00,992.24,996.00,0.315393,2.892312,0.051222,0.102032,-0.398718,-0.353902,-2.757467e-15
6,1483299360,2017-01-01 19:36:00,996.00,999.36,994.70,999.36,0.732966,2.899923,0.060376,0.105688,-0.398718,-0.380960,-2.757467e-15
7,1483303020,2017-01-01 20:37:00,999.36,1000.00,999.36,1000.00,0.134029,2.915189,0.067177,0.102424,-0.398718,-0.392986,-2.757467e-15
8,1483303080,2017-01-01 20:38:00,1000.00,1002.00,996.25,1001.98,0.427591,2.917786,0.073032,0.097325,-0.349975,-0.403676,-2.757467e-15
9,1483305360,2017-01-01 21:16:00,1000.00,1004.88,999.40,1004.00,0.435482,2.925299,0.078110,0.091199,-0.398718,-0.355572,-2.757467e-15


In [6]:
predict_data = data.copy().drop(["Open","Close","High","Low"],axis=1)
max_days = 5
target_range = 2
for i in range(1,max_days):#2jours
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
    predict_data[["Variation-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

In [7]:
predict_data

,Unix,Date,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,...,MACD_H-2,Variation-3,RSI-3,MACD-3,MACD_H-3,Variation-4,RSI-4,MACD-4,MACD_H-4,Target
0,1483289880,2017-01-01 16:58:00,0.554800,2.877294,0.041725,0.096336,-0.398718,-0.353234,-2.757467e-15,1.748401,...,0.060230,0.857599,2.588866,0.007401,0.043285,-0.215342,3.072177,0.000444,0.031253,1
1,1483294500,2017-01-01 18:15:00,0.315393,2.892312,0.051222,0.102032,-0.398718,-0.353902,-2.757467e-15,0.554800,...,0.082427,1.385308,2.705708,0.017140,0.060230,0.857599,2.588866,0.007401,0.043285,1
2,1483299360,2017-01-01 19:36:00,0.732966,2.899923,0.060376,0.105688,-0.398718,-0.380960,-2.757467e-15,0.315393,...,0.096336,1.748401,2.808421,0.030164,0.082427,1.385308,2.705708,0.017140,0.060230,1
3,1483303020,2017-01-01 20:37:00,0.134029,2.915189,0.067177,0.102424,-0.398718,-0.392986,-2.757467e-15,0.732966,...,0.102032,0.554800,2.877294,0.041725,0.096336,1.748401,2.808421,0.030164,0.082427,1
4,1483303080,2017-01-01 20:38:00,0.427591,2.917786,0.073032,0.097325,-0.349975,-0.403676,-2.757467e-15,0.134029,...,0.105688,0.315393,2.892312,0.051222,0.102032,0.554800,2.877294,0.041725,0.096336,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206009,1646098440,2022-03-01 01:34:00,0.296195,0.381483,1.930740,-2.075293,-0.398718,0.654442,-1.173104e+00,0.124220,...,-2.118649,-1.377128,-0.195675,2.468978,-1.899447,0.263767,-0.384912,2.755311,-1.432747,0
206010,1646099400,2022-03-01 01:50:00,0.068895,0.967749,1.821356,-1.948001,-0.398718,0.046630,-1.184698e+00,0.296195,...,-2.182438,0.166687,0.025907,2.241247,-2.118649,-1.377128,-0.195675,2.468978,-1.899447,0
206011,1646099520,2022-03-01 01:52:00,-0.800713,0.508509,1.600107,-2.140442,1.490217,-0.403676,-1.200815e+00,0.068895,...,-2.075293,0.124220,0.718270,2.055929,-2.182438,0.166687,0.025907,2.241247,-2.118649,0
206012,1646099880,2022-03-01 01:58:00,-0.686825,-0.306142,1.310527,-2.474145,1.847879,-0.403676,-1.193196e+00,-0.800713,...,-1.948001,0.296195,0.381483,1.930740,-2.075293,0.124220,0.718270,2.055929,-2.182438,1


In [8]:
predict_data = predict_data[[i % int(max_days) == 0 for i in range(len(predict_data))]]

In [9]:
predict_data

,Unix,Date,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,...,MACD_H-2,Variation-3,RSI-3,MACD-3,MACD_H-3,Variation-4,RSI-4,MACD-4,MACD_H-4,Target
0,1483289880,2017-01-01 16:58:00,0.554800,2.877294,0.041725,0.096336,-0.398718,-0.353234,-2.757467e-15,1.748401,...,0.060230,0.857599,2.588866,0.007401,0.043285,-0.215342,3.072177,0.000444,0.031253,1
5,1483305360,2017-01-01 21:16:00,0.435482,2.925299,0.078110,0.091199,-0.398718,-0.355572,-2.757467e-15,0.427591,...,0.102424,0.732966,2.899923,0.060376,0.105688,0.315393,2.892312,0.051222,0.102032,0
10,1483348320,2017-01-02 09:12:00,2.646727,2.587762,0.091749,0.062459,-0.398718,-0.198733,-2.757467e-15,0.473151,...,0.052395,0.482494,2.380996,0.078431,0.059258,-0.931778,2.932234,0.077961,0.072550,1
15,1483368000,2017-01-02 14:40:00,1.859745,1.058707,0.097589,0.028397,-0.398718,-0.195560,-2.757467e-15,3.496477,...,0.023129,0.491214,2.526744,0.105674,0.072585,0.412667,2.569060,0.099328,0.069886,0
20,1483389240,2017-01-02 20:34:00,1.275329,0.349417,0.061248,-0.056609,-0.398718,-0.249510,4.726359e-02,1.111274,...,-0.043392,-2.576908,0.324350,0.089061,-0.004730,-0.574650,0.843618,0.098419,0.024883,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205990,1646088120,2022-02-28 22:42:00,-0.141182,2.325574,5.947438,4.250882,0.141224,-0.403676,1.727393e+00,-0.808921,...,6.009634,0.685592,2.558514,5.335524,6.105957,0.866130,2.210527,4.765973,5.759630,0
205995,1646089200,2022-02-28 23:00:00,1.457458,1.082850,4.259548,-1.989485,-0.398718,-0.403676,2.599016e+00,-1.878375,...,0.503727,-2.150257,1.826483,5.682701,1.989837,0.737182,2.476841,6.012577,3.572041,0
206000,1646092980,2022-03-01 00:03:00,-0.404795,-0.285172,3.185063,-2.062797,-0.398718,-0.403676,1.462009e+00,1.527078,...,-3.148237,0.116068,0.275278,3.366533,-3.366619,-1.111924,0.831576,3.773226,-2.870932,1
206005,1646096100,2022-03-01 00:55:00,0.263767,-0.384912,2.755311,-1.432747,-0.398718,1.153352,-2.569900e-01,0.166457,...,-1.613315,0.630394,-0.419476,3.044598,-1.631565,-0.210806,-0.514340,3.074506,-1.941087,0


In [10]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date", "Unix", "Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date", "Unix", "Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [12]:
algo = "MLP"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(100,200,100),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

Iteration 1, loss = 0.65398021
Iteration 2, loss = 0.64336664
Iteration 3, loss = 0.63996147
Iteration 4, loss = 0.63833888
Iteration 5, loss = 0.63697008
Iteration 6, loss = 0.63595846
Iteration 7, loss = 0.63490123
Iteration 8, loss = 0.63386533
Iteration 9, loss = 0.63308808
Iteration 10, loss = 0.63230100
Iteration 11, loss = 0.63149277
Iteration 12, loss = 0.63073962
Iteration 13, loss = 0.62990224
Iteration 14, loss = 0.62882980
Iteration 15, loss = 0.62831250
Iteration 16, loss = 0.62773880
Iteration 17, loss = 0.62650033
Iteration 18, loss = 0.62577357
Iteration 19, loss = 0.62499218
Iteration 20, loss = 0.62419966
Iteration 21, loss = 0.62347343
Iteration 22, loss = 0.62242316
Iteration 23, loss = 0.62162843
Iteration 24, loss = 0.62097936
Iteration 25, loss = 0.61968942
Iteration 26, loss = 0.61909913
Iteration 27, loss = 0.61817302
Iteration 28, loss = 0.61718370
Iteration 29, loss = 0.61618977
Iteration 30, loss = 0.61551841
Iteration 31, loss = 0.61454742
Iteration 32, los

c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(100, 200, 100), verbose=True)

In [50]:
model.score(test_data,test_labels)

0.5018050541516246

In [51]:
model.score(train_data,train_labels)

1.0

In [35]:
from sklearn.metrics import confusion_matrix

In [53]:
preds = model.predict(test_data)

In [54]:
conf = confusion_matrix(test_labels,preds)
conf

array([[53, 72],
       [66, 86]], dtype=int64)

In [55]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.44537815126050423

In [56]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5443037974683544

## Cross Validation

In [19]:
def cross_validation_loop(predict_data, model_name):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        if model_name == "MLP":
            model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        elif model_name == "CAT":
            model = CatBoostClassifier(iterations=100,depth=12,verbose=False)
        elif model_name == "RF":
            model = RandomForestClassifier(n_estimators=100,verbose=False)
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [20]:
#scores,TPs,TNs = cross_validation_loop(predict_data,"CAT")

In [24]:
from statistics import mean

In [50]:
print("CatBoost score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5067452023560707 || TP moyen : 0.5411620348700146 || TN moyen : 0.4619500842660272


In [22]:
scores,TPs,TNs = cross_validation_loop(predict_data,"RF")

Score : 0.51985559566787 || TP : 0.5694444444444444 || TN : 0.46616541353383456
Score : 0.49458483754512633 || TP : 0.5194805194805194 || TN : 0.4634146341463415
Score : 0.5054151624548736 || TP : 0.5465116279069767 || TN : 0.4380952380952381
Score : 0.48375451263537905 || TP : 0.5136986301369864 || TN : 0.45038167938931295
Score : 0.4693140794223827 || TP : 0.4722222222222222 || TN : 0.4639175257731959
Score : 0.48736462093862815 || TP : 0.525 || TN : 0.4358974358974359
Score : 0.51985559566787 || TP : 0.5280898876404494 || TN : 0.5050505050505051
Score : 0.49458483754512633 || TP : 0.5611510791366906 || TN : 0.427536231884058
Score : 0.49458483754512633 || TP : 0.5384615384615384 || TN : 0.4380165289256198
Score : 0.4657039711191336 || TP : 0.4899328859060403 || TN : 0.4375
Score : 0.4693140794223827 || TP : 0.5205479452054794 || TN : 0.4122137404580153
Score : 0.5306859205776173 || TP : 0.5714285714285714 || TN : 0.46788990825688076
Score : 0.4584837545126354 || TP : 0.5 || TN : 0.4

In [25]:
print("Random Forest score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

Random Forest score moyen : 0.5040851225536767 || TP moyen : 0.538149033546309 || TN moyen : 0.4600308310035195


In [26]:
scores,TPs,TNs = cross_validation_loop(predict_data,"MLP")

Score : 0.51985559566787 || TP : 0.5675675675675675 || TN : 0.46511627906976744
Score : 0.48014440433212996 || TP : 0.5068493150684932 || TN : 0.45038167938931295
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5234657039711191 || TP : 0.5496688741721855 || TN : 0.49206349206349204
Score : 0.4657039711191336 || TP : 0.46308724832214765 || TN : 0.46875
Score : 0.5342960288808665 || TP : 0.5734265734265734 || TN : 0.4925373134328358
Score : 0.5270758122743683 || TP : 0.541095890410959 || TN : 0.5114503816793893
Score : 0.5054151624548736 || TP : 0.5657894736842105 || TN : 0.432
Score : 0.48014440433212996 || TP : 0.5266666666666666 || TN : 0.4251968503937008
Score : 0.5090252707581228 || TP : 0.5302013422818792 || TN : 0.484375
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5451263537906137 || TP : 0.5933333333333334 || TN : 0.4881889763779528
Score : 0.4729241877256318 || TP : 0.5131578947368421 || TN : 0.424
Score : 0.534296028

In [27]:
print("MLP score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

MLP score moyen : 0.5078852365570967 || TP moyen : 0.5443692036536304 || TN moyen : 0.4653102613033886


# Predicting 2021 with 2019 and 2020

In [13]:
data_19_20 = predict_data.loc[(predict_data["Unix"] >= 1546300800) & (predict_data["Unix"] < 1609459200)]

In [14]:
data_19_20

,Unix,Date,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,...,MACD_H-2,Variation-3,RSI-3,MACD-3,MACD_H-3,Variation-4,RSI-4,MACD-4,MACD_H-4,Target
65820,1546302180,2019-01-01 00:23:00,-0.354484,-1.245238,-0.236549,-0.055408,-0.398718,-0.281579,0.457788,-0.071650,...,-0.139614,-0.277736,-1.926709,-0.260609,-0.205089,-0.192869,-1.111457,-0.249924,-0.221202,0
65825,1546320660,2019-01-01 05:31:00,0.301055,-0.897523,-0.184058,0.082221,-0.398718,-0.386973,1.397454,-0.286567,...,0.035341,-0.033102,-1.206462,-0.223926,-0.005377,-0.090221,-1.324541,-0.230678,-0.028900,1
65830,1546334940,2019-01-01 09:29:00,-0.296867,0.986963,-0.087818,0.182897,-0.123183,-0.403676,1.392224,-0.255245,...,0.189212,-0.307540,1.013803,-0.138121,0.159270,1.123476,1.513749,-0.157139,0.136572,1
65835,1546350780,2019-01-01 13:53:00,-2.748450,0.269770,-0.036135,0.141009,0.152822,-0.403676,0.505168,2.088269,...,0.153205,-0.010447,0.786964,-0.065762,0.168733,0.242643,1.275254,-0.075824,0.177853,0
65840,1546367880,2019-01-01 18:38:00,1.811076,-0.541441,-0.100090,-0.068050,-0.398718,-0.403676,-0.246927,-2.320738,...,0.015172,0.015071,-0.072923,-0.057021,0.041626,-0.433161,0.122194,-0.048218,0.081004,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128055,1609451220,2020-12-31 21:47:00,0.614193,0.465378,0.472662,-0.166923,0.215672,-0.403676,-1.039676,-1.191303,...,0.141941,0.415630,0.838443,0.556910,0.069152,-0.157644,0.650842,0.531826,0.003980,0
128060,1609453320,2020-12-31 22:22:00,-0.137603,0.283425,0.205872,-0.400510,-0.398718,0.252574,-1.148936,1.082064,...,-0.698221,0.302536,-0.353325,0.288482,-0.529499,-0.993172,-0.420051,0.355062,-0.442920,1
128065,1609455600,2020-12-31 23:00:00,-0.856507,-0.328855,0.230783,-0.209564,2.148178,-0.403676,-1.209783,-0.030146,...,0.004917,0.225599,0.237612,0.299106,-0.032848,0.438641,0.187830,0.247216,-0.211666,1
128070,1609456980,2020-12-31 23:23:00,-0.135191,0.174578,0.102134,-0.342790,0.487915,-0.403676,-0.925617,-0.497795,...,-0.153683,0.084809,-0.337166,0.204388,-0.193962,0.510833,-0.284855,0.212484,-0.215807,1


In [15]:
data_2021 = predict_data.loc[predict_data["Unix"] >= 1609459200]

In [16]:
data_2021

,Unix,Date,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,...,MACD_H-2,Variation-3,RSI-3,MACD-3,MACD_H-3,Variation-4,RSI-4,MACD-4,MACD_H-4,Target
128080,1609459740,2021-01-01 00:09:00,-0.994752,-1.287569,-0.351093,-0.634216,1.767476,-0.403676,0.605752,-0.448905,...,-0.292895,0.336757,-0.496276,-0.161497,-0.333098,0.338735,-0.877619,-0.194767,-0.525750,1
128085,1609461300,2021-01-01 00:35:00,0.365410,-0.044396,-0.305421,-0.020863,-0.398718,0.047632,1.339805,0.009357,...,-0.369562,0.509791,-0.836591,-0.419319,-0.549317,0.307825,-1.119572,-0.419689,-0.687841,0
128090,1609462080,2021-01-01 00:48:00,0.199901,0.172645,-0.084460,0.413467,-0.398718,-0.184035,0.679584,0.129158,...,0.232894,0.110781,-0.228173,-0.239459,0.144886,-0.233381,-0.381856,-0.276439,0.059530,1
128095,1609463460,2021-01-01 01:11:00,0.701738,2.669881,0.398695,1.079190,-0.398718,1.065329,0.350583,0.393957,...,0.712666,0.522024,0.594391,0.049508,0.592610,-0.025851,-0.085693,-0.038096,0.452719,1
128100,1609464420,2021-01-01 01:27:00,0.429970,2.294756,0.911450,1.118222,-0.398718,-0.021351,1.310834,-0.300307,...,1.407646,-0.136100,2.498107,0.733442,1.466696,0.872627,2.704607,0.597402,1.386056,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205990,1646088120,2022-02-28 22:42:00,-0.141182,2.325574,5.947438,4.250882,0.141224,-0.403676,1.727393,-0.808921,...,6.009634,0.685592,2.558514,5.335524,6.105957,0.866130,2.210527,4.765973,5.759630,0
205995,1646089200,2022-02-28 23:00:00,1.457458,1.082850,4.259548,-1.989485,-0.398718,-0.403676,2.599016,-1.878375,...,0.503727,-2.150257,1.826483,5.682701,1.989837,0.737182,2.476841,6.012577,3.572041,0
206000,1646092980,2022-03-01 00:03:00,-0.404795,-0.285172,3.185063,-2.062797,-0.398718,-0.403676,1.462009,1.527078,...,-3.148237,0.116068,0.275278,3.366533,-3.366619,-1.111924,0.831576,3.773226,-2.870932,1
206005,1646096100,2022-03-01 00:55:00,0.263767,-0.384912,2.755311,-1.432747,-0.398718,1.153352,-0.256990,0.166457,...,-1.613315,0.630394,-0.419476,3.044598,-1.631565,-0.210806,-0.514340,3.074506,-1.941087,0


In [17]:
split_mode = "STATIC"

#if split_mode == "RAND":
#    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
if split_mode == "STATIC":
    train_data = data_19_20.drop(["Date", "Unix", "Target"],axis=1)
    train_labels = data_19_20["Target"]
    test_data = data_2021.drop(["Date", "Unix", "Target"],axis=1)
    test_labels = data_2021["Target"]

In [18]:
%load_ext tensorboard

In [19]:
shape = (len(test_data.columns),)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [20]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [21]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback], batch_size=64)

Epoch 1/10
195/195 [==============================] - 23s 111ms/step - loss: 0.6957 - accuracy: 0.5122 - precision: 0.5238 - recall: 0.7010 - val_loss: 0.6935 - val_accuracy: 0.4982 - val_precision: 0.5030 - val_recall: 0.4278
Epoch 2/10
195/195 [==============================] - 20s 101ms/step - loss: 0.6924 - accuracy: 0.5223 - precision: 0.5271 - recall: 0.8089 - val_loss: 0.6997 - val_accuracy: 0.5044 - val_precision: 0.5044 - val_recall: 1.0000
Epoch 3/10
195/195 [==============================] - 20s 101ms/step - loss: 0.6925 - accuracy: 0.5167 - precision: 0.5224 - recall: 0.8437 - val_loss: 0.6938 - val_accuracy: 0.5045 - val_precision: 0.5045 - val_recall: 0.9986
Epoch 4/10
195/195 [==============================] - 20s 101ms/step - loss: 0.6919 - accuracy: 0.5231 - precision: 0.5259 - recall: 0.8593 - val_loss: 0.6935 - val_accuracy: 0.5021 - val_precision: 0.5036 - val_recall: 0.9030
Epoch 5/10
195/195 [==============================] - 19s 98ms/step - loss: 0.6912 - accurac

In [38]:
test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

#print('\nTest accuracy:', test_acc)

3816/3816 - 23s - loss: 0.6122 - accuracy: 0.7459 - precision: 0.7216 - recall: 0.8129 - 23s/epoch - 6ms/step


ValueError: too many values to unpack (expected 2)

In [40]:
time = str(datetime.date.today())
#model.save('./models/1m_200_2000_200_y2019_2020_'+time)

INFO:tensorflow:Assets written to: ./models/1m_200_2000_200_y2019_20202022-09-29\assets
